In [ ]:
Ux = np.random.default_rng().uniform(low=-15, high=15, size=500)
Uy = np.random.default_rng().uniform(low=-15, high=15, size=500)
Uz = np.random.default_rng().uniform(low=-15, high=15, size=500)

In [ ]:
def calc_max_covariance(self, df: pd.DataFrame, colx: str, coly: str, lags: int = 10) -> [float, int]:
    """
    Find maximum covariance between two variables
    :param df: Pandas DataFrame containing the data
    :param colx: DataFrame column with x variable
    :param coly: DataFrame column with y variable
    :param lags: number of lags to search over; default is 10; larger number requires more time
    :return: maximum covariance between two variables, lag number of max covariance
    """
    dfcov = []
    for i in np.arange(-1 * lags, lags):
        df[f"{coly}_{i}"] = df[coly].shift(i)
        dfcov.append(df[[colx, f"{coly}_{i}"]].cov().loc[colx, f"{coly}_{i}"])
        # print(i,df[[colx, f"{coly}_{i}"]].cov().loc[colx, f"{coly}_{i}"])
        df = df.drop([f"{coly}_{i}"], axis=1)

    abscov = np.abs(dfcov)
    maxabscov = np.max(abscov)
    try:
        maxlagindex = np.where(abscov == maxabscov)[0][0]
        lagno = maxlagindex - lags
        maxcov = dfcov[maxlagindex]
    except IndexError:
        lagno = 0
        maxcov = dfcov[10]
    return maxcov, lagno

In [ ]:
data = np.linspace(0,50,1000)
x = np.sin(data)
y = np.cos(data)

#plt.plot(data, x)
#plt.plot(data, y)

print(np.cov(x,y,bias=True))

mca = MCA(np.stack([data,x], axis=1), np.stack([data,y], axis=1))
mca.solve(complexify=True,period=100)
#plt.plot(mca.temporal_amplitude()['left'])
#plt.plot(mca.temporal_amplitude()['right'])
plt.plot(mca.temporal_phase()['left'])
#plt.plot(mca.temporal_phase()['right'])
mca.singular_values(n=100)

In [ ]:
mca.correlation_matrix()

In [ ]:
Ux = np.random.default_rng().uniform(low=-15, high=15, size=500)
Uy = np.random.default_rng().uniform(low=-15, high=15, size=500)
Uz = np.random.default_rng().uniform(low=-15, high=15, size=500)

In [ ]:
from numba import njit, prange
import numpy as np

@njit(parallel=True)
def shadow_correction(Ux, Uy, Uz):
    """Correction for flow distortion of CSAT sonic anemometer from Horst and others (2015) based on work by Kaimal

    :param Ux: Longitudinal component of the wind velocity (m s-1); aka u
    :param Uy: Lateral component of the wind velocity (m s-1); aka v
    :param Uz: Vertical component of the wind velocity (m s-1); aka w
    :return: corrected wind components
    """

    # Rotation Matrix to Align with Path Coordinate System of Transducers
    h = [0.25, 0.4330127018922193, 0.8660254037844386,
         -0.5, 0.0, 0.8660254037844386,
         0.25, -0.4330127018922193, 0.8660254037844386]

    # Inverse of the Rotation Matrix
    hinv = [0.6666666666666666, -1.3333333333333333, 0.6666666666666666,
            1.1547005383792517, 0.0, -1.1547005383792517,
            0.38490017945975047, 0.38490017945975047, 0.38490017945975047]

    iteration = 0

    while iteration < 4:

        Uxh = h[0] * Ux + h[1] * Uy + h[2] * Uz
        Uyh = h[3] * Ux + h[4] * Uy + h[5] * Uz
        Uzh = h[6] * Ux + h[7] * Uy + h[8] * Uz

        scalar = np.sqrt(Ux ** 2. + Uy ** 2. + Uz ** 2.)

        Theta1 = np.arccos(np.abs(h[0] * Ux + h[1] * Uy + h[2] * Uz) / scalar)
        Theta2 = np.arccos(np.abs(h[3] * Ux + h[4] * Uy + h[5] * Uz) / scalar)
        Theta3 = np.arccos(np.abs(h[6] * Ux + h[7] * Uy + h[8] * Uz) / scalar)

        #  Adjustment Factors for Each Component
        # Adjust for the Shadowing Effects

        Uxa = Uxh / (0.84 + 0.16 * np.sin(Theta1))
        Uya = Uyh / (0.84 + 0.16 * np.sin(Theta2))
        Uza = Uzh / (0.84 + 0.16 * np.sin(Theta3))

        # Transform the Winds Components Back to the CSAT Coordinate System.
        # These are the Corrected Velocities.

        Uxc = hinv[0] * Uxa + hinv[1] * Uya + hinv[2] * Uza
        Uyc = hinv[3] * Uxa + hinv[4] * Uya + hinv[5] * Uza
        Uzc = hinv[6] * Uxa + hinv[7] * Uya + hinv[8] * Uza

        Ux = Uxc
        Uy = Uyc
        Uz = Uzc

        iteration += 1

    return Uxc, Uyc, Uzc

%timeit shadow_correction(Ux, Uy, Uz)
shadow_correction(Ux, Uy, Uz)

In [ ]:
from numba import jit, int32

@jit
def shadow_correction(Ux, Uy, Uz):
    
    # Rotation Matrix to Align with Path Coordinate System of Transducers
    h = [0.25, 0.4330127018922193, 0.8660254037844386, 
                  -0.5, 0.0, 0.8660254037844386, 
                  0.25, -0.4330127018922193, 0.8660254037844386]

    # Inverse of the Rotation Matrix
    hinv = [0.6666666666666666, -1.3333333333333333, 0.6666666666666666, 
                  1.1547005383792517, 0.0, -1.1547005383792517, 
                  0.38490017945975047, 0.38490017945975047, 0.38490017945975047]


    Uxh = h[0]*Ux + h[1]*Uy + h[2]*Uz
    Uyh = h[3]*Ux + h[4]*Uy + h[5]*Uz
    Uzh = h[6]*Ux + h[7]*Uy + h[8]*Uz

    scalar = np.sqrt(Ux ** 2. + Uy ** 2. + Uz ** 2.)

    Theta1 = np.arccos(np.abs(h[0]*Ux + h[1]*Uy + h[2]*Uz) / scalar)
    Theta2 = np.arccos(np.abs(h[3]*Ux + h[4]*Uy + h[5]*Uz) / scalar)
    Theta3 = np.arccos(np.abs(h[6]*Ux + h[7]*Uy + h[8]*Uz) / scalar)

    #  Adjustment Factors for Each Component

    c1 = 1. / (0.84 + 0.16*np.sin(Theta1))
    c2 = 1. / (0.84 + 0.16*np.sin(Theta2))
    c3 = 1. / (0.84 + 0.16*np.sin(Theta3))

    # Adjust for the Shadowing Effects

    Uxa=Uxh*c1
    Uya=Uyh*c2
    Uza=Uzh*c3

    # Transform the Winds Components Back to the CSAT Coordinate System.
    # These are the Corrected Velocities.

    Uxc = hinv[0]*Uxa + hinv[1]*Uya + hinv[2]*Uza
    Uyc = hinv[3]*Uxa + hinv[4]*Uya + hinv[5]*Uza
    Uzc = hinv[6]*Uxa + hinv[7]*Uya + hinv[8]*Uza

    return Uxc, Uyc, Uzc
#Uxx=Uxc
#Uyy=Uyc
#Uzz=Uzc
%timeit shadow_correction(Ux, Uy, Uz)
shadow_correction(Ux, Uy, Uz)

In [ ]:
https://link.springer.com/article/10.1007/s10546-015-0010-3

@jit
def shadow_correction(Ux, Uy, Uz):
    
    # Rotation Matrix to Align with Path Coordinate System of Transducers
    h = [0.25, 0.4330127018922193, 0.8660254037844386, 
                  -0.5, 0.0, 0.8660254037844386, 
                  0.25, -0.4330127018922193, 0.8660254037844386]

    # Inverse of the Rotation Matrix
    hinv = [0.6666666666666666, -1.3333333333333333, 0.6666666666666666, 
                  1.1547005383792517, 0.0, -1.1547005383792517, 
                  0.38490017945975047, 0.38490017945975047, 0.38490017945975047]


    Uxh = h[0]*Ux + h[1]*Uy + h[2]*Uz
    Uyh = h[3]*Ux + h[4]*Uy + h[5]*Uz
    Uzh = h[6]*Ux + h[7]*Uy + h[8]*Uz

    scalar = np.sqrt(Ux ** 2. + Uy ** 2. + Uz ** 2.)

    Theta1 = np.arccos(np.abs(h[0]*Ux + h[1]*Uy + h[2]*Uz) / scalar)
    Theta2 = np.arccos(np.abs(h[3]*Ux + h[4]*Uy + h[5]*Uz) / scalar)
    Theta3 = np.arccos(np.abs(h[6]*Ux + h[7]*Uy + h[8]*Uz) / scalar)

    #  Adjustment Factors for Each Component
    # Adjust for the Shadowing Effects

    Uxa = Uxh / (0.84 + 0.16*np.sin(Theta1))
    Uya = Uyh / (0.84 + 0.16*np.sin(Theta2))
    Uza = Uzh / (0.84 + 0.16*np.sin(Theta3))

    # Transform the Winds Components Back to the CSAT Coordinate System.
    # These are the Corrected Velocities.

    Uxc = hinv[0]*Uxa + hinv[1]*Uya + hinv[2]*Uza
    Uyc = hinv[3]*Uxa + hinv[4]*Uya + hinv[5]*Uza
    Uzc = hinv[6]*Uxa + hinv[7]*Uya + hinv[8]*Uza

    return Uxc, Uyc, Uzc
#Uxx=Uxc
#Uyy=Uyc
#Uzz=Uzc
shadow_correction(Ux, Uy, Uz)

In [ ]:
def fix_csat(Ux, Uy, Uz):

    CSAT3Inverse = [[-0.5, 0, 0.8660254037844386],
                    [0.25, 0.4330127018922193, 0.8660254037844386],
                    [0.25, -0.4330127018922193, 0.8660254037844386]]
    CSAT3Transform = [[-1.3333333333333, 0.66666666666666, 0.66666666666666],
                      [0, 1.1547005383792, -1.1547005383792],
                      [0.3849001794597, 0.3849001794597, 0.3849001794597]]

    u = {}
    u[0] = CSAT3Inverse[0][0] * Ux + CSAT3Inverse[0][1] * Uy + CSAT3Inverse[0][2] * Uz
    u[1] = CSAT3Inverse[1][0] * Ux + CSAT3Inverse[1][1] * Uy + CSAT3Inverse[1][2] * Uz
    u[2] = CSAT3Inverse[2][0] * Ux + CSAT3Inverse[2][1] * Uy + CSAT3Inverse[2][2] * Uz

    scalar = (Ux ** 2. + Uy ** 2. + Uz ** 2.) ** 0.5

    u[0] = u[0] / (0.68 + 0.32 * np.sin(np.arccos(u[0] / scalar)))
    u[1] = u[1] / (0.68 + 0.32 * np.sin(np.arccos(u[1] / scalar)))
    u[2] = u[2] / (0.68 + 0.32 * np.sin(np.arccos(u[2] / scalar)))

    Ux_out = CSAT3Transform[0][0] * u[0] + CSAT3Transform[0][1] * u[1] + CSAT3Transform[0][2] * u[2]
    Uy_out = CSAT3Transform[1][0] * u[0] + CSAT3Transform[1][1] * u[1] + CSAT3Transform[1][2] * u[2]
    Uz_out = CSAT3Transform[2][0] * u[0] + CSAT3Transform[2][1] * u[1] + CSAT3Transform[2][2] * u[2]

    return Ux_out, Uy_out, Uz_out

fix_csat(Ux, Uy, Uz)

In [ ]:
#@numba.njit#(forceobj=True)
def fix_csat(Ux, Uy, Uz):

    CSAT3Inverse = [[-0.5, 0, 0.8660254037844386],
                    [0.25, 0.4330127018922193, 0.8660254037844386],
                    [0.25, -0.4330127018922193, 0.8660254037844386]]
    CSAT3Transform = [[-1.3333333333333, 0.66666666666666, 0.66666666666666],
                      [0, 1.1547005383792, -1.1547005383792],
                      [0.3849001794597, 0.3849001794597, 0.3849001794597]]

    Ux_out = []
    Uy_out = []
    Uz_out = []

    for i in range(len(Ux)):
        u = {}
        u[0] = CSAT3Inverse[0][0] * Ux[i] + CSAT3Inverse[0][1] * Uy[i] + CSAT3Inverse[0][2] * Uz[i]
        u[1] = CSAT3Inverse[1][0] * Ux[i] + CSAT3Inverse[1][1] * Uy[i] + CSAT3Inverse[1][2] * Uz[i]
        u[2] = CSAT3Inverse[2][0] * Ux[i] + CSAT3Inverse[2][1] * Uy[i] + CSAT3Inverse[2][2] * Uz[i]

        scalar = (Ux[i] ** 2. + Uy[i] ** 2. + Uz[i] ** 2.) ** 0.5

        u[0] = u[0] / (0.68 + 0.32 * np.sin(np.arccos(u[0] / scalar)))
        u[1] = u[1] / (0.68 + 0.32 * np.sin(np.arccos(u[1] / scalar)))
        u[2] = u[2] / (0.68 + 0.32 * np.sin(np.arccos(u[2] / scalar)))

        Ux_out.append(CSAT3Transform[0][0] * u[0] + CSAT3Transform[0][1] * u[1] + CSAT3Transform[0][2] * u[2])
        Uy_out.append(CSAT3Transform[1][0] * u[0] + CSAT3Transform[1][1] * u[1] + CSAT3Transform[1][2] * u[2])
        Uz_out.append(CSAT3Transform[2][0] * u[0] + CSAT3Transform[2][1] * u[1] + CSAT3Transform[2][2] * u[2])

    return Ux_out, Uy_out, Uz_out

fix_csat(Ux, Uy, Uz)

In [ ]:
def calc_Td(E):
    """
    Dew point equation
    :param E: Water vapour pressure at saturation
    :return:
    """
    c0 = 207.98233
    c1 = -20.156028
    c2 = 0.46778925
    c3 = -0.0000092288067
    d0 = 1
    d1 = -0.13319669
    d2 = 0.0056577518
    d3 = -0.000075172865
    lne = np.log(E)
    return (c0 + c1 * lne + c2 * lne ** 2 + c3 * lne ** 3) / (d0 + d1 * lne + d2 * lne ** 2 + d3 * lne ** 3)

def calc_Q(P, E):
    return (0.622 * E) / (P - 0.378 * E)

def calc_E(pV, T, Rv):
    return pV * Rv * T

def calc_L(Ust, Tsa, Uz_Ta):
    #removed negative sign
    return -1*(Ust ** 3) * Tsa / (9.8 * 0.4 * Uz_Ta)

#@numba.njit#(forceobj=True)
def calc_Tsa(Ts, P, pV, Rv=461.51):
    """
    Calculate the average sonic temperature
    :param Ts:
    :param P:
    :param pV:
    :param Rv:
    :return:
    """
    E = pV * Rv * Ts
    return -0.01645278052 * (
                -500 * P - 189 * E + np.sqrt(250000 * P ** 2 + 128220 * E * P + 35721 * E ** 2)) / pV / Rv


#@numba.njit#(forceobj=True)
def tetens(t, a=0.611, b=17.502, c=240.97):
    """Tetens formula for computing the
    saturation vapor pressure of water from temperature; eq. 3.8

    :param t: temperature (C)
    :param a: constant (kPa)
    :param b: constant (dimensionless)
    :param c: constant (C)
    :return: saturation vapor pressure (kPa)
    """
    return a * np.exp((b * t) / (t + c))

def convert_CtoK(T):
    """
    Convert Celcius to Kelvin
    :param T: Temperature in Celcius degrees
    :return: Temperature in Kelvin
    """
    return T + 273.16

#@numba.jit(forceobj=True)
def calc_Es(T: float) -> float:
    """
    Saturation Vapor Pressure Equation
    :param T: Water temperature in Celcius
    :return: Saturation Vapor Pressure (Pa)
    """
    g0 = -2836.5744
    g1 = -6028.076559
    g2 = 19.54263612
    g3 = -0.02737830188
    g4 = 0.000016261698
    g5 = 0.00000000070229056
    g6 = -0.00000000000018680009
    g7 = 2.7150305
    
    T = convert_CtoK(T)
    return np.exp(
        g0 * T ** (-2) + g1 * T ** (-1) + g2 + g3 * T + g4 * T ** 2 + g5 * T ** 3 + g6 * T ** 4 + g7 * np.log(T))

In [ ]:
print(tetens(15))
print(calc_Es(15))

In [ ]:
from dataclasses import dataclass, field

In [ ]:
class Constants:
    c0: float = 207.98233
    c1: float = -20.156028
    c2: float = 0.46778925
    c3: float = -0.0000092288067
    d0: float = 1
    d1: float = -0.13319669
    d2: float = 0.0056577518
    d3: float = -0.000075172865
        
    g0: float = -2836.5744
    g1: float = -6028.076559
    g2: float = 19.54263612
    g3: float = -0.02737830188
    g4: float = 0.000016261698
    g5: float = 0.00000000070229056
    g6: float = -0.00000000000018680009
    g7: float = 2.7150305



In [ ]:
const = Constants()
const.g0